In [1]:
!pip install torch transformers 

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
!pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv('D:\\Downloads_D\\Final_semtiment_document(2).csv', nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)


# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  710.8491199016571
Accuracy: 0.955


In [20]:
label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels.csv', index=False)


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 1002), nrows=999)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('predicted_labels2.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  722.5253391265869
Accuracy: 0.914914914914915


In [9]:
label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels2.csv', index=False)


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 2001), nrows=999)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('predicted_labels3.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  1646.4429984092712
Accuracy: 0.8898898898898899


In [11]:
label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels3.csv', index=False)

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 3000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('predicted_labels4.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  2946.95468378067
Accuracy: 0.902


In [14]:
label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels4.csv', index=False)

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 4000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('predicted_labels5.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  979.1526069641113
Accuracy: 0.891


In [15]:
label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels5.csv', index=False)


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 5000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels6.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  3271.3673996925354
Accuracy: 0.915


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 6000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels7.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  3689.446438074112
Accuracy: 0.926


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 7000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels8.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  2341.2914152145386
Accuracy: 0.89


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 8000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels9.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  2968.4429030418396
Accuracy: 0.886


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 9000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels10.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  1366.7848510742188
Accuracy: 0.949


In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 10000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels11.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  2019.7031331062317
Accuracy: 0.899


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 11000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels12.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  3288.959051847458
Accuracy: 0.948


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 12000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels13.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  3026.19251871109
Accuracy: 0.927


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

# Record start time
start_time = time.time()

# Load data
data = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv",skiprows=range(1, 13000), nrows=1000)
labels = data['Sentiment Labels'].tolist()  # Assuming the column name is 'Sentiment Labels'
reviews = data['Text_new'].tolist()

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize data
encodings = tokenizer(reviews, truncation=True, padding=True, max_length=512)

# Encode labels into numerical values
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = [label_map[label.lower()] for label in labels]

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloader
dataset = ReviewDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)  # 3 labels: positive, negative, neutral

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # number of epochs
    model.train()
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}", unit="batch", leave=False):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the model with its configuration
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

# Load the model with its configuration
model = RobertaForSequenceClassification.from_pretrained('sentiment_model_RoBERTa')

# Initialize loader without tqdm for evaluation
loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Evaluate the model using GPU
model.eval()
predictions = []
true_labels = []
for batch in loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Calculate and print evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')


label_map_inverse = {v: k for k, v in label_map.items()}  # Define inverse mapping

# Add predicted labels to the original dataframe
data['Predicted Labels'] = [label_map_inverse[label] for label in predictions]

# Save the dataframe to a new CSV file
data.to_csv('D:\\Downloads_D\\predicted_labels14.csv', index=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Menaka\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Time required to fine-tune:  2449.1167488098145
Accuracy: 0.931


In [8]:
import pandas as pd


file_names = ["D:\Downloads_D\predicted_labels.csv",
              "D:\Downloads_D\predicted_labels2.csv",
              "D:\Downloads_D\predicted_labels3.csv",
              "D:\Downloads_D\predicted_labels4.csv",
              "D:\Downloads_D\predicted_labels5.csv",
              "D:\Downloads_D\predicted_labels6.csv",
              "D:\Downloads_D\predicted_labels7.csv",
              "D:\Downloads_D\predicted_labels8.csv", 
              "D:\Downloads_D\predicted_labels9.csv", 
              "D:\Downloads_D\predicted_labels10.csv", 
              "D:\Downloads_D\predicted_labels11.csv",
              "D:\Downloads_D\predicted_labels12.csv",
              "D:\Downloads_D\predicted_labels13.csv",
              "D:\Downloads_D\predicted_labels14.csv",
             ]


dfs = []


for file in file_names:
    df = pd.read_csv(file)
    dfs.append(df)


combined_df = pd.concat(dfs, ignore_index=True)


combined_df.to_csv("D:\\Downloads_D\\novelty_14.csv", index=False)

In [16]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Read the CSV files
df = pd.read_csv("D:\\Downloads_D\\novelty_14.csv")
rd = pd.read_csv("D:\\Downloads_D\\Final_semtiment_document(2).csv")

# Lowercase the actual labels only
actual_labels = df['Sentiment Labels'].str.lower().tolist()
roberta_labels = df['Predicted Labels'].tolist()

# Calculate accuracy
accuracy = accuracy_score(actual_labels, roberta_labels)

print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.92
